In [1]:
# 라이브러리 추가
import requests; from urllib.parse import urlparse
import pandas as pd
import geopandas

In [3]:
# 카카오맵 주소 검색 API: https://developers.kakao.com/docs/restapi/local#주소-검색
address = "강원 원주시 단구로 171"
url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"})
json_obj = result.json()
print(json_obj)

{'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}, 'documents': [{'y': '37.33698555530167', 'road_address': {'region_2depth_name': '원주시', 'main_building_no': '171', 'building_name': '국립공원연구원', 'address_name': '강원 원주시 단구로 171', 'undergroun_yn': 'N', 'zone_no': '26441', 'x': '127.9477445454879', 'underground_yn': 'N', 'road_name': '단구로', 'y': '37.33698555530167', 'region_1depth_name': '강원', 'region_3depth_name': '명륜동', 'sub_building_no': ''}, 'address_type': 'ROAD_ADDR', 'address_name': '강원 원주시 단구로 171', 'address': {'main_address_no': '242', 'sub_adderss_no': '2', 'h_code': '4213054100', 'sub_address_no': '2', 'zip_code': '220947', 'address_name': '강원 원주시 명륜동 242-2', 'x': '127.94787953303228', 'main_adderss_no': '242', 'y': '37.33695203787654', 'region_3depth_h_name': '명륜1동', 'region_2depth_name': '원주시', 'region_1depth_name': '강원', 'b_code': '4213010600', 'region_3depth_name': '명륜동', 'mountain_yn': 'N'}, 'x': '127.9477445454879'}]}


In [4]:
# 검색 결과 정렬
list = []
for document in json_obj['documents']:
    val = [document['road_address']['building_name'], document['address_name'], document['y'], document['x']]
    list.append(val)
    
df = pd.DataFrame(list, columns = ['building_name', 'address_name', 'lat', 'lon'])
df

,building_name,address_name,lat,lon
0,국립공원연구원,강원 원주시 단구로 171,37.33698555530167,127.9477445454879


In [5]:
# 주소 목록 xlsx 파일 읽기
df = pd.read_excel('D:/GEODATA/OFF_ADR.xlsx', sheet_name='Sheet1')
df

""


In [ ]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"})
    json_obj = result.json()
    for document in json_obj['documents']:
        val = [document['road_address']['building_name'], document['address_name'], document['y'], document['x']]
    return val

In [ ]:
# 주소 검색 함수 적용
address_to_latlon("강원 원주시 단구로 171")

In [ ]:
# 주소 검색 함수 반복 적용
list = []
for address in df['도로명주소']:
    list.append(address_to_latlon(address))
    
df1 = pd.DataFrame(list, columns = ['building_name', 'address_name', 'lat', 'lon'])
df1.head()

In [ ]:
# 컬럼 값 수정
df1['office_name'] = df['지자체명']
df1.head()

In [ ]:
# xlsx 파일로 저장
df1.to_excel('D:/GEODATA/NLPRK_OFF_LATLON.xlsx')

In [ ]:
# Shapefile로 저장
geometry=geopandas.points_from_xy(df1.lon.astype(float), df1.lat.astype(float))
gdf = geopandas.GeoDataFrame(df1, geometry=geometry)
gdf.crs= "+init=epsg:4326"
gdf.to_file('D:/GEODATA/NLPRK_OFF_LATLON.shp', driver='ESRI Shapefile', encoding='utf-8')